In [1]:
import pyomo
import cplex

#### The notebook takes as input a python pyomo code for a non-linear program (with unified variables) and creates an RLT(Reformulation Linearization Technique) relaxation to give valid bounds. 

The function below gets an constraint in the form of mathematical expression (expr). It then constructs lifting variables and uses them to form a linearized version of the constraint. Then it returns the linearized constraint.

In [2]:
from sympy import expand
def rlt_cons(c, expr , variables, Indices, rhs):
    #print to check the simplified expression
    #Now that you have a simplified expression, navigate the expression and make changes to it by introducing the lifting variables.

    # Following code creates a list of variables var_list
    var_str = ""
    var_list = []
    signs = []    # This list stores the signs of each term of the constraint
    for j in range(len(expr)):
        if expr[j]=="+" or expr[j]=="-":
            if j==0:
                signs.append(expr[j])
                continue
            else:
                var_list.append(var_str[:-1].strip())
                var_str = ""
                signs.append(expr[j])
                continue
        if  j==len(expr)-1:
            var_list.append(var_str.strip()+expr[j])
        var_str += expr[j]
    if expr[0]!="-":
        signs = ['+'] + signs

    # 2. Store the linearized variable

    VariablesLocal = []
    coeff = []          # This is to store coefficients of every term
    for v in var_list:
        v_ctr = 0
        if len(v)>0 and v[0].isnumeric():
            for j in range(len(v)):
                if v[j]=="*":
                    v_ctr = j+1
                    break
                if j==len(v)-1:
                    v_ctr = j+1
            if v_ctr == len(v):
                coeff.append(v)
            else:
                coeff.append(int(v[0:v_ctr-1]))
        elif len(v)>0:
            coeff.append(1)
        X = "X"
        ind = []
        for j in range(v_ctr, len(v)):
            if v[j]=="x":
                k = j+1
                l = ""
                while k<len(v) and v[k].isnumeric():
                    l += v[k]
                    k+=1
                ind.append(int(l))
                pow = 0
                if k<len(v)-1 and v[k]=="*" and v[k+1]=="*":
                    pow = int(v[k+2])
                for p2 in range(pow-1):
                    ind.append(int(l))
            
        ind.sort()
        for j in ind:
            X+="_%d"%j
        final = ""
        if len(ind)!=0:
            VariablesLocal.append(X)
        else:
            VariablesLocal.append("")
        # Now updating the variables and Indices dictionary and list respectively
        if X not in variables and len(ind)!=0:
            variables.append(X)
            Indices[X] = ind
    # Now returning the new constraint which is linearized
    new_c = ""
    for i1 in range(len(VariablesLocal)):
        new_c +=  signs[i1]+str(coeff[i1])
        if len(VariablesLocal[i1])>1:
            new_c+="*" + "m."+VariablesLocal[i1]
        else:
            new_c+=VariablesLocal[i1]
    new_c += c[rhs:]
    return new_c

In [3]:
import itertools
# This function reads a model, RLT variable and creates a bound constraint for that variable
def create_bound_cons(mdl, v, Indices):
    var_list =  list(mdl.component_objects(Var))
    L = len(Indices)
    facts = []   # This stores a list of all bound factor constraints..... 
    for comb in itertools.product([0,1], repeat = L):
        fact = ""
        for i in range(len(comb)):
            if i!=0:
                fact += "*"
            if comb[i]==1:
                fact += "(%d - m.x%d)"%(var_list[Indices[i]-1].bounds[1],Indices[i])
            else:
                fact += "(m.x%d - (%d))"%(Indices[i],var_list[Indices[i]-1].bounds[0])
        fact += " >= 0"
        facts.append(fact)
    return facts
    

## The code below creates a pyomo model based on the original and lifted variables for a RLT formulation. The optimal solution of this gives a valid bound to the original non-linear programming problem.

In [5]:
# MINLP written by GAMS Convert at 08/29/22 13:55:14
#
# Equation counts
#     Total        E        G        L        N        X        C        B
#        78       12       66        0        0        0        0        0
#
# Variable counts
#                  x        b        i      s1s      s2s       sc       si
#     Total     cont   binary  integer     sos1     sos2    scont     sint
#        37       37        0        0        0        0        0        0
# FX      0
#
# Nonzero counts
#     Total    const       NL
#       498       66      432
#
# Reformulation has removed 1 variable and 1 equation

from pyomo.environ import *
from itertools import combinations
#from sympy import *



model = m = ConcreteModel()

m.x1 = Var(within=Reals, bounds=(-2,2), initialize=-1.313011472)
m.x2 = Var(within=Reals, bounds=(-2,2), initialize=1.373066832)
m.x3 = Var(within=Reals, bounds=(-2,2), initialize=0.201501424)
m.x4 = Var(within=Reals, bounds=(-2,2), initialize=-0.795448384)
m.x5 = Var(within=Reals, bounds=(-2,2), initialize=-0.831151532)
m.x6 = Var(within=Reals, bounds=(-2,2), initialize=-1.103788532)
m.x7 = Var(within=Reals, bounds=(-2,2), initialize=-0.600677984)
m.x8 = Var(within=Reals, bounds=(-2,2), initialize=1.425081388)
m.x9 = Var(within=Reals, bounds=(-2,2), initialize=-1.731545108)
m.x10 = Var(within=Reals, bounds=(-2,2), initialize=0.000842675999999987)
m.x11 = Var(within=Reals, bounds=(-2,2), initialize=1.992470508)
m.x12 = Var(within=Reals, bounds=(-2,2), initialize=0.314933512)
m.x13 = Var(within=Reals, bounds=(-2,2), initialize=1.964532156)
m.x14 = Var(within=Reals, bounds=(-2,2), initialize=1.049001868)
m.x15 = Var(within=Reals, bounds=(-2,2), initialize=-1.477230068)
m.x16 = Var(within=Reals, bounds=(-2,2), initialize=0.558875036)
m.x17 = Var(within=Reals, bounds=(-2,2), initialize=-1.361928544)
m.x18 = Var(within=Reals, bounds=(-2,2), initialize=-0.999677868)
m.x19 = Var(within=Reals, bounds=(-2,2), initialize=0.675714436)
m.x20 = Var(within=Reals, bounds=(-2,2), initialize=-0.258574476)
m.x21 = Var(within=Reals, bounds=(-2,2), initialize=-0.561198936)
m.x22 = Var(within=Reals, bounds=(-2,2), initialize=-0.594234528)
m.x23 = Var(within=Reals, bounds=(-2,2), initialize=-1.47403364)
m.x24 = Var(within=Reals, bounds=(-2,2), initialize=-1.399592848)
m.x25 = Var(within=Reals, bounds=(-2,2), initialize=0.3564546)
m.x26 = Var(within=Reals, bounds=(-2,2), initialize=1.323571248)
m.x27 = Var(within=Reals, bounds=(-2,2), initialize=-1.076737048)
m.x28 = Var(within=Reals, bounds=(-2,2), initialize=0.66293784)
m.x29 = Var(within=Reals, bounds=(-2,2), initialize=1.103430424)
m.x30 = Var(within=Reals, bounds=(-2,2), initialize=-0.785366092)
m.x31 = Var(within=Reals, bounds=(-2,2), initialize=-1.558030836)
m.x32 = Var(within=Reals, bounds=(-2,2), initialize=0.00953946399999994)
m.x33 = Var(within=Reals, bounds=(-2,2), initialize=-1.359308952)
m.x34 = Var(within=Reals, bounds=(-2,2), initialize=1.489849244)
m.x35 = Var(within=Reals, bounds=(-2,2), initialize=-0.93954182)
m.x36 = Var(within=Reals, bounds=(-2,2), initialize=-0.856742712)
m.x37 = Var(within=Reals, bounds=(None,None), initialize=0)




# The following is a string of constraints

inp = open("InputFile.txt",'r')
st = inp.read()
str_cons = st[st.find("m.e1 "):]
str_cons = str_cons.strip()



# Following is the code for creating a list of constraints

current_cons = ""
constraint_list = []
for i in range(len(str_cons)):
    # If constraint is changing append
    if i<len(str_cons) - 2 and str_cons[i]==")" and str_cons[i+1]=="\n" and str_cons[i+2]=="m":
        constraint_list.append(current_cons+str_cons[i])
        current_cons = ""
    else:
        if str_cons[i]!="\n":
            current_cons += str_cons[i]


# Now replace an expression with its simplified form....


variables = []
Indices = {}
cons_str = []
cons_no = [0]

var_list =  model.component_objects(Var)
for v in var_list:
    this_var_ind = ""
    for d in range(1, len(str(v))):
        this_var_ind += str(v)[d]
    var_ind = int(this_var_ind)
    if "X_%d"%var_ind not in variables and v.bounds[0] is not None and v.bounds[1] is not None:
        cons_str.append("m.X_%d>=%d"%(var_ind, v.bounds[0]))
        cons_str.append("m.X_%d<=%d"%(var_ind, v.bounds[1]))
        cons_no[0] = cons_no[0] + 2
        variables.append("X_%d"%var_ind)
        Indices["X_%d"%var_ind] = [var_ind]

# Now navigate to all the constraints and save the variables and linearize the constraint

for i in range(len(constraint_list)):
    c = constraint_list[i]
    print("printing orig",c)
    #print(c, "  ....this constraint")
    pos = c.find("expr")
    pos = pos+6
    #Now make a command for opening up the brackets
    simplify_command = "expr = expand(" 
    rhs_pos = -1
    for j in  range(pos, len(c)):
        if c[j]=="=" or c[j]==">" or c[j]=="<":
            rhs_pos = j
            break
        simplify_command += c[j]
    simplify_command+=")"
    exec(simplify_command)
    expr = str(expr)
    str_app = rlt_cons(c[:-1], expr, variables, Indices, rhs_pos)
    print("printing new",str_app)
    cons_str.append(str_app)
    cons_no[0] = cons_no[0] + 1



indices_list = []
print(Indices)
for v in variables:
    indices_list.append(Indices[v])

# Now introduce all the variables that aren't yet defined in the model
for v in variables:
    for i in range(len(Indices[v])):
        for comb in list(combinations(Indices[v],i)):
            try_var = "X"
            for c in list(comb):
                try_var += "_%d"%c
            if try_var not in variables:
                variables.append(try_var)
                Indices[try_var] = list(comb)
                indices_list.append(try_var)

print("Variables:",variables)
for v in variables:
    if len(Indices[v])>1:
        facts = create_bound_cons(m, v, Indices[v])
        facts2 = []
        for f in facts:
            simp_com = "expr = expand("
            rhs_pos = -1
            for j in  range(len(f)):
                if f[j]=="=" or f[j]==">" or f[j]=="<":
                    rhs_pos = j
                    break
                simp_com += f[j]
            simp_com += ")"
            exec(simp_com)
            expr = str(expr)
            expr2 = rlt_cons(f, expr, variables, Indices, rhs_pos)
            cons_str.append(expr2)
            cons_no[0] = cons_no[0] + 1

print(cons_str[-1])

printing orig m.e1 = Constraint(expr= m.x1**2 + m.x2**2 + m.x3**2 == 4)
printing new +1*m.X_1_1+1*m.X_2_2+1*m.X_3_3== 4
printing orig m.e2 = Constraint(expr= m.x4**2 + m.x5**2 + m.x6**2 == 4)
printing new +1*m.X_4_4+1*m.X_5_5+1*m.X_6_6== 4
printing orig m.e3 = Constraint(expr= m.x7**2 + m.x8**2 + m.x9**2 == 4)
printing new +1*m.X_7_7+1*m.X_8_8+1*m.X_9_9== 4
printing orig m.e4 = Constraint(expr= m.x10**2 + m.x11**2 + m.x12**2 == 4)
printing new +1*m.X_10_10+1*m.X_11_11+1*m.X_12_12== 4
printing orig m.e5 = Constraint(expr= m.x13**2 + m.x14**2 + m.x15**2 == 4)
printing new +1*m.X_13_13+1*m.X_14_14+1*m.X_15_15== 4
printing orig m.e6 = Constraint(expr= m.x16**2 + m.x17**2 + m.x18**2 == 4)
printing new +1*m.X_16_16+1*m.X_17_17+1*m.X_18_18== 4
printing orig m.e7 = Constraint(expr= m.x19**2 + m.x20**2 + m.x21**2 == 4)
printing new +1*m.X_19_19+1*m.X_20_20+1*m.X_21_21== 4
printing orig m.e8 = Constraint(expr= m.x22**2 + m.x23**2 + m.x24**2 == 4)
printing new +1*m.X_22_22+1*m.X_23_23+1*m.X_24_24

C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'x1**2 + x2**2 + x3**2'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  return sympify(e).expand(deep=deep, modulus=modulus, **hints)
C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define symp

printing new +1*m.X_10_10-2*m.X_10_13+1*m.X_11_11-2*m.X_11_14+1*m.X_12_12-2*m.X_12_15+1*m.X_13_13+1*m.X_14_14+1*m.X_15_15-4*m.X_37>= 0
printing orig m.e44 = Constraint(expr= (m.x10 - m.x16)**2 + (m.x11 - m.x17)**2 + (m.x12 -    m.x18)**2 - 4 * m.x37 >= 0)
printing new +1*m.X_10_10-2*m.X_10_16+1*m.X_11_11-2*m.X_11_17+1*m.X_12_12-2*m.X_12_18+1*m.X_16_16+1*m.X_17_17+1*m.X_18_18-4*m.X_37>= 0
printing orig m.e45 = Constraint(expr= (m.x10 - m.x19)**2 + (m.x11 - m.x20)**2 + (m.x12 -    m.x21)**2 - 4 * m.x37 >= 0)
printing new +1*m.X_10_10-2*m.X_10_19+1*m.X_11_11-2*m.X_11_20+1*m.X_12_12-2*m.X_12_21+1*m.X_19_19+1*m.X_20_20+1*m.X_21_21-4*m.X_37>= 0
printing orig m.e46 = Constraint(expr= (m.x10 - m.x22)**2 + (m.x11 - m.x23)**2 + (m.x12 -    m.x24)**2 - 4 * m.x37 >= 0)
printing new +1*m.X_10_10-2*m.X_10_22+1*m.X_11_11-2*m.X_11_23+1*m.X_12_12-2*m.X_12_24+1*m.X_22_22+1*m.X_23_23+1*m.X_24_24-4*m.X_37>= 0
printing orig m.e47 = Constraint(expr= (m.x10 - m.x25)**2 + (m.x11 - m.x26)**2 + (m.x12 -    m.x2

C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'(x10 - x16)**2 + (x11 - x17)**2 + (x12 - x18)**2 - 4*x37'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  return sympify(e).expand(deep=deep, modulus=modulus, **hints)
C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sy

C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'(x2 + 2)*(2 - x17)'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  return sympify(e).expand(deep=deep, modulus=modulus, **hints)
C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify

...

C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'(x9 + 2)*(2 - x33)'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  return sympify(e).expand(deep=deep, modulus=modulus, **hints)
C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify

C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'(2 - x14)*(x17 + 2)'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  return sympify(e).expand(deep=deep, modulus=modulus, **hints)
C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympif

C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'(2 - x17)*(2 - x35)'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  return sympify(e).expand(deep=deep, modulus=modulus, **hints)
C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympif

+1*m.X_30_36-2*m.X_30-2*m.X_36+4>= 0


C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'(2 - x28)*(x31 + 2)'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  return sympify(e).expand(deep=deep, modulus=modulus, **hints)
C:\Users\schoudhary66\Anaconda3\lib\site-packages\sympy\core\function.py:2833: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympif

In [6]:
print(variables)
print(Indices)

for v in variables:
    if Indices[v]!=[]:
        stcmd1 = "m.%s = Var(within = Reals)"%v
        exec(stcmd1)

['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17', 'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26', 'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35', 'X_36', 'X_1_1', 'X_2_2', 'X_3_3', 'X_4_4', 'X_5_5', 'X_6_6', 'X_7_7', 'X_8_8', 'X_9_9', 'X_10_10', 'X_11_11', 'X_12_12', 'X_13_13', 'X_14_14', 'X_15_15', 'X_16_16', 'X_17_17', 'X_18_18', 'X_19_19', 'X_20_20', 'X_21_21', 'X_22_22', 'X_23_23', 'X_24_24', 'X_25_25', 'X_26_26', 'X_27_27', 'X_28_28', 'X_29_29', 'X_30_30', 'X_31_31', 'X_32_32', 'X_33_33', 'X_34_34', 'X_35_35', 'X_36_36', 'X_1_4', 'X_2_5', 'X_3_6', 'X_37', 'X_1_7', 'X_2_8', 'X_3_9', 'X_1_10', 'X_2_11', 'X_3_12', 'X_1_13', 'X_2_14', 'X_3_15', 'X_1_16', 'X_2_17', 'X_3_18', 'X_1_19', 'X_2_20', 'X_3_21', 'X_1_22', 'X_2_23', 'X_3_24', 'X_1_25', 'X_2_26', 'X_3_27', 'X_1_28', 'X_2_29', 'X_3_30', 'X_1_31', 'X_2_32', 'X_3_33', 'X_1_34', 'X_2_35', 'X_3_36', 'X_4_7', '

In [7]:
for i in range(cons_no[0]):
    print(cons_str[i])
    stcmd = "m.e%d = Constraint( expr = %s )"%(i+1, cons_str[i])
    print(stcmd)
    exec(stcmd)
m.obj = Objective(sense=maximize, expr= m.X_37)

m.X_1>=-2
m.e1 = Constraint( expr = m.X_1>=-2 )
m.X_1<=2
m.e2 = Constraint( expr = m.X_1<=2 )
m.X_2>=-2
m.e3 = Constraint( expr = m.X_2>=-2 )
m.X_2<=2
m.e4 = Constraint( expr = m.X_2<=2 )
m.X_3>=-2
m.e5 = Constraint( expr = m.X_3>=-2 )
m.X_3<=2
m.e6 = Constraint( expr = m.X_3<=2 )
m.X_4>=-2
m.e7 = Constraint( expr = m.X_4>=-2 )
m.X_4<=2
m.e8 = Constraint( expr = m.X_4<=2 )
m.X_5>=-2
m.e9 = Constraint( expr = m.X_5>=-2 )
m.X_5<=2
m.e10 = Constraint( expr = m.X_5<=2 )
m.X_6>=-2
m.e11 = Constraint( expr = m.X_6>=-2 )
m.X_6<=2
m.e12 = Constraint( expr = m.X_6<=2 )
m.X_7>=-2
m.e13 = Constraint( expr = m.X_7>=-2 )
m.X_7<=2
m.e14 = Constraint( expr = m.X_7<=2 )
m.X_8>=-2
m.e15 = Constraint( expr = m.X_8>=-2 )
m.X_8<=2
m.e16 = Constraint( expr = m.X_8<=2 )
m.X_9>=-2
m.e17 = Constraint( expr = m.X_9>=-2 )
m.X_9<=2
m.e18 = Constraint( expr = m.X_9<=2 )
m.X_10>=-2
m.e19 = Constraint( expr = m.X_10>=-2 )
m.X_10<=2
m.e20 = Constraint( expr = m.X_10<=2 )
m.X_11>=-2
m.e21 = Constraint( expr = m.X_11>

-1*m.X_17_35+2*m.X_17-2*m.X_35+4>= 0
m.e899 = Constraint( expr = -1*m.X_17_35+2*m.X_17-2*m.X_35+4>= 0 )
-1*m.X_17_35-2*m.X_17+2*m.X_35+4>= 0
m.e900 = Constraint( expr = -1*m.X_17_35-2*m.X_17+2*m.X_35+4>= 0 )
+1*m.X_17_35-2*m.X_17-2*m.X_35+4>= 0
m.e901 = Constraint( expr = +1*m.X_17_35-2*m.X_17-2*m.X_35+4>= 0 )
+1*m.X_18_36+2*m.X_18+2*m.X_36+4>= 0
m.e902 = Constraint( expr = +1*m.X_18_36+2*m.X_18+2*m.X_36+4>= 0 )
-1*m.X_18_36+2*m.X_18-2*m.X_36+4>= 0
m.e903 = Constraint( expr = -1*m.X_18_36+2*m.X_18-2*m.X_36+4>= 0 )
-1*m.X_18_36-2*m.X_18+2*m.X_36+4>= 0
m.e904 = Constraint( expr = -1*m.X_18_36-2*m.X_18+2*m.X_36+4>= 0 )
+1*m.X_18_36-2*m.X_18-2*m.X_36+4>= 0
m.e905 = Constraint( expr = +1*m.X_18_36-2*m.X_18-2*m.X_36+4>= 0 )
+1*m.X_19_22+2*m.X_19+2*m.X_22+4>= 0
m.e906 = Constraint( expr = +1*m.X_19_22+2*m.X_19+2*m.X_22+4>= 0 )
-1*m.X_19_22+2*m.X_19-2*m.X_22+4>= 0
m.e907 = Constraint( expr = -1*m.X_19_22+2*m.X_19-2*m.X_22+4>= 0 )
-1*m.X_19_22-2*m.X_19+2*m.X_22+4>= 0
m.e908 = Constraint( expr =

In [8]:
solver = SolverFactory('cplex')
results = solver.solve(m)
inp.close()

In [9]:
print(results)


Problem: 
- Name: tmp3bn6r74b
  Lower bound: 8.0
  Upper bound: 8.0
  Number of objectives: 1
  Number of constraints: 1074
  Number of variables: 269
  Number of nonzeros: 3315
  Sense: maximize
Solver: 
- Status: ok
  User time: 0.01
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 8.0000000000e+00
  Error rc: 0
  Time: 0.053588151931762695
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
m.pprint()
print(len(variables))